In [ ]:
import numpy as np
import math

In [ ]:
def activesetquadraticprogramming(x, d, G, ce, be, ci, bi, tol, W=[]):
    w, v = np.linalg.eig(G)
    hasnegigenval = np.any(w < 0)
    n = len(x)
    ni = len(ci)
    ne = len(ce)
    ce = np.array(ce)
    ci = np.array(ci)
    bi = np.array(bi)
    be = np.array(be)
    if ni == 0:
        ci = np.zeros((0, n))
    if ne == 0:
        ce = np.zeros((0, n))
    eq = np.matmul(ce, x) - be
    iq = np.matmul(ci, x) - bi
    if np.all(np.abs(eq) < tol) and np.all(iq > -tol):
        if len(W) != ni:
            W = iq < tol
    else:
        c = np.concatenate((np.zeros(2 * n), np.ones(ni + 2 * ne)))
        A = np.concatenate((np.concatenate((-ci, ci, np.eye(ni), np.zeros((ni, 2 * ne))), 1),
                            np.concatenate((ce, -ce, np.zeros((ne, ni)), np.eye(ne), np.zeros((ne, ne))), 1),
                            np.concatenate((-ce, ce, np.zeros((ne, ni + ne)), np.eye(ne)), 1)), 0)
        b = np.concatenate((-bi, be, -be))
        x, opt, res = simplextableau(c, A, b)
        x = x[0:n] - x[n:2 * n]
        W = ci * x - bi < tol
        if res != 'solution found':
            return x, W, [], 'infeasible constraints'
    for k in range(4 * ni + 4):
        na = np.sum(W)
        A = np.concatenate((ce, ci[W, :]), 0)
        b = np.concatenate((be, bi[W]), 0)
        c = np.matmul(A, x) - b
        g = d + np.matmul(G, x)
        K = np.concatenate(
            (np.concatenate((G, np.transpose(A)), 1), np.concatenate((A, np.zeros((ne + na, ne + na))), 1)),
            0)
        npl = np.linalg.solve(K, np.concatenate((g, c), 0))
        p = -npl[0:n]

        if hasnegigenval:
            cdirnegcurv = False
            Q, R = np.linalg.qr(np.transpose(A), 'complete')
            Z = Q[:, na:]
            if not Z.shape[1] == 0:
                w, v = np.linalg.eig(np.matmul(np.matmul(np.transpose(Z), G), Z))
                if np.min(w) < 0:
                    dd = np.matmul(Z, v[:, np.argmin(w)])
                    p = -1e4 * np.sign(np.dot(g, dd)) * dd
                    cdirnegcurv = True
        if np.all(np.abs(p) < tol):
            lambdaW = npl[n + ne:n + ne + na]
            l = np.zeros(ni)
            l[W] = lambdaW
            if np.all(lambdaW >= -tol):
                return x, W, l, k, 'Solution found'
            else:
                mlw = np.sort(lambdaW[lambdaW < 0])
                ri = mlw[0] == l
                if 'ac' in locals() and np.all(ri == ac) and len(mlw) > 1:
                    ri = mlw[np.ceil((len(mlw) - 1) * np.rand())] == l
                W[ri] = False
        else:
            cip = np.matmul(ci, p)
            bicixcip = (bi - np.matmul(ci, x)) / cip
            findminof = bicixcip[np.invert(W) & (cip < 0)]
            if len(findminof > 0):
                alpha = np.min(findminof)
            else:
                alpha = 1
            if alpha > 1:
                alpha = 1
            if hasnegigenval and alpha == 1 and cdirnegcurv:
                print('Problem unbounded, did not find a constraint in a negative curvature direction')
                return x, W, [], k, 'Problem unbounded, did not find a constraint in a negative curvature direction'
            x = x + alpha * p
            if alpha < 1:
                ac = (alpha == bicixcip) & np.invert(W) & (cip < 0)
                W = W | ac
    print('Problem unbounded, did not find a constraint in a negative curvature direction')
    return x, W, l, k, 'Problem unbounded, did not find a constraint in a negative curvature direction'


def simplextableau(c, A, b):
    tol = 1e-14
    res = ''
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    m, n = A.shape
    Ni = np.array(range(n - m))
    Bi = np.array(range(m)) + n - m
    x = np.zeros((n, 1))
    xB = np.array(b)
    combs = math.factorial(n) / (math.factorial(m) * math.factorial(n - m))
    for k in range(4 * m):
        l = np.linalg.solve(A[:, Bi], c[Bi])
        sN = c[Ni] - np.matmul(np.transpose(A[:, Ni]), l)
        sm = np.min(sN)
        qq = np.argmin(sN)
        q = Ni[qq]
        xm = np.min(xB)
        p = np.argmin(xB)
        mu = np.minimum(sm, xm)
        if mu >= -tol:
            res = 'solution found'
            break
        if mu == sm:
            a = A[:, q]
            p = np.argmax(a)
            phi = A[p, q]
            if phi <= tol:
                res = 'primal infeasible or unbounded'
                break
        else:
            sigma = A[p, Ni]
            qq = np.argmin(sigma)
            q = Ni[qq]
            phi = A[p, q]
            if phi >= -tol:
                res = 'dual infeasible or unbounded'
        xB[p] = xB[p] / phi
        A[p, :] = A[p, :] / phi
        oi = list(range(m))
        oi.remove(p)
        xB[oi] = xB[oi] - A[oi, q] * xB[p]
        A[oi, :] = A[oi, :] - np.multiply.outer(A[oi, q], A[p, :])
        Ni[Ni == q] = Bi[p]
        Bi[p] = q
    x[Bi, 0] = xB
    opt = np.dot(c, x)
    if len(res) == 0:
        res = 'Iterations exceed maximum number, cycling may be happening.'
    return x, opt[0], res

In [ ]:
def convert_to_standard_QP(M, y, x):
    m, n = M.shape[0], 2 * M.shape[0]
    # objective function = 1/2 * x^T * G * x + c^T * x = 1/2*(x^T M^T M x - 2 y^T M x (+y^T y))
    G = M.T @ M
    c = 2 * M.T @ y
    ai = np.zeros((m, n))
    bi = np.ones(m)
    ai[:, :n // 2] = np.eye(m)

    return G, c, ai, bi

In [ ]:
# draw M array of shape (m, 2m)
M = np.random.rand(2, 4)
print(M)

In [ ]:
# draw y such that operator Norm ||M|| <= ||y||_2
y = np.random.rand(2, 1)
y += 1
print(y)

In [ ]:
print(np.linalg.norm(M, ord=2))
print(np.linalg.norm(y, ord=2))

In [ ]:
# draw x
x = np.random.rand(4, 1)
print(x)

In [ ]:
# is f feasible for the problem?
print(np.linalg.norm(x, ord=1) <= 1)

In [ ]:
G, c, ai, bi = convert_to_standard_QP(M, y, x)
print(G)
print(c)
print(ai)
print(bi)

In [ ]:
# solve the problem
x, W, l, k, res = activesetquadraticprogramming(x, c, G, [], [], ai, bi, 1e-6)

In [ ]:
zeros = np.zeros((2,4))
M_tilde = np.array([[1,1,0,0],[0,0,1,1]])

M = np.array([[M_tilde, zeros, zeros, zeros, zeros],
             [zeros, M_tilde, zeros, zeros, zeros],
             [zeros, zeros, M_tilde, zeros, zeros],
             [zeros, zeros, zeros, M_tilde, zeros],
             [zeros, zeros, zeros, zeros, M_tilde]])

# make a 10 by 20 matrix of this form
M = M.reshape(10, 20)

In [ ]:
print(M)

In [ ]:
M = np.block([  [M_tilde, np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4))],
                [np.zeros((2,4)), M_tilde, np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4))],
                [np.zeros((2,4)), np.zeros((2,4)), M_tilde, np.zeros((2,4)), np.zeros((2,4))],
                [np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4)), M_tilde, np.zeros((2,4))],
                [np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4)), np.zeros((2,4)), M_tilde]])

In [ ]:
print(M.shape)
print(M)

In [ ]:
print(M.T@M)